In [46]:
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import re
import requests
import seaborn as sns

from db import create_connection
from db import (
    articles,
    services,
    urls
)

In [47]:
pd.set_option('max_colwidth', 400)

In [48]:
connection = create_connection()

In [49]:
df = pd.read_csv('se_lublin.csv')
df.head(1)

,Unnamed: 0,publication_date,publication_month,publication_day,author,title,url,koronawirus_in_text,koronawirus_in_title,question_mark,exclamation_mark,all_words,koronawirus_anywhere,koronawirus_anywhere_count
0,0,2020-04-15 10:42:00,2020-04,2020-04-15,marek targoński,Lubelskie: AŻ 25 NOWYCH ZAKAŻEŃ. Koronawirus atakuje z ogromną siłą! [NOWE DANE],https://www.se.pl/lublin/lubelskie-az-25-nowych-zakazen-koronawirus-atakuje-z-ogromna-sila-nowe-dane-aa-xMvU-hSfD-xgA8.html,13,1,0,2,160,14,1


In [55]:
name_urls = df[['url']].where(df['publication_month'] == name).dropna()

In [50]:
def url_to_transcript(url):
    page = requests.get(url).text
    bs = BeautifulSoup(page, "lxml")
    container = bs.find(class_="main-content")
    article = container.find("article")
    texts = article.find_all("p")
    articles = []
    for t in texts:
        article = t.text
        articles.append(article)
    return articles

In [51]:
data = {}

In [53]:
names = ["2020-01", "2020-02", "2020-03", "2020-04", "2020-05", "2020-06", "2020-07", "2020-08"]

In [69]:
for name in names:
    name_urls = df[['url']].where(df['publication_month'] == name).dropna()
    text = []
    for key, url in name_urls.items():
        for k, v in url.items():
            text.append(url_to_transcript(v))
    text_string = ''.join([str(item) for item in text])
    data[name] = text_string


In [71]:
for i, name in enumerate(names):
    with open("transcripts/" + name + ".txt", "wb") as file:
        pickle.dump(data[name], file)

In [72]:
data = {}
for i, name in enumerate(names):
    with open("transcripts/" + name + ".txt", "rb") as file:
        data[name] = pickle.load(file)

In [73]:
data.keys()

dict_keys(['2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08'])

In [74]:
def combine_text(list_of_text):
    combined_text = ''.join(list_of_text)
    return combined_text

In [75]:
data_combined = {key: [combine_text(value)] for (key, value) in data.items()}

In [79]:
pd.set_option('max_colwidth', 150)
data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['transcript']
data_df = data_df.sort_index()
data_df

,transcript
2020-01,['Prawie trzy tysiące e-zwolnień wystawiają dziennie lekarze z Lubelszczyzny. Nowy kreator do ich wypełniania od Zakładu Ubezpieczeń Społecznych m...
2020-02,['Dobre informacje z lubelskiego lotniska. W czerwcu ruszy nowe połączenie z Portu Lotniczego Lublin. Ze Świdnika polecimy do Kolonii/Bonn. Połącz...
2020-03,"['To ten sam mężczyzna, u którego wykryto chorobę we wtorek. Do tej pory przebywał w szpitalu w Bełżycach, ale w czwartek przetransportowano go do..."
2020-04,['Ministerstwo Zdrowia podało w środę przed południem nowe dane z frontu walki z epidemią koronawirusa w Polsce! Pięć ofiar śmiertelnych i ponad 2...
2020-05,"['Brał właśnie do rąk szkło, by skosztować produkty destylacji... Policjanci z Białej Podlaskiej (woj. lubelskie) wkroczyli do domu 62-letniego mi..."
2020-06,"['Zamiast wziąć głęboki oddech i policzyć do dziesięciu, aby się uspokoić, wybrał wariant najgłupszy z możliwych. 21-latek z Zamościa, którego do ..."
2020-07,"['Tęgo popił, chciał więc nie gorzej podjeść! 49-letni kierowca elektrycznego meleksa pędził po Puławach chodnikami i ścieżkami rowerowymi do miej..."
2020-08,"['Kostry. Potężna wściekłość i chęć ukarania mężczyzny, który krzywdził jego siostrę, dosłownie oślepiła Piotra N. (21 l.). Kiedy spotkał go wiecz..."


In [80]:
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    #text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [81]:
data_clean = pd.DataFrame(data_df.transcript.apply(round1))
data_clean

,transcript
2020-01,prawie trzy tysiące ezwolnień wystawiają dziennie lekarze z lubelszczyzny nowy kreator do ich wypełniania od zakładu ubezpieczeń społecznych może ...
2020-02,dobre informacje z lubelskiego lotniska w czerwcu ruszy nowe połączenie z portu lotniczego lublin ze świdnika polecimy do koloniibonn połączenie u...
2020-03,to ten sam mężczyzna u którego wykryto chorobę we wtorek do tej pory przebywał w szpitalu w bełżycach ale w czwartek przetransportowano go do samo...
2020-04,ministerstwo zdrowia podało w środę przed południem nowe dane z frontu walki z epidemią koronawirusa w polsce pięć ofiar śmiertelnych i ponad now...
2020-05,brał właśnie do rąk szkło by skosztować produkty destylacji policjanci z białej podlaskiej woj lubelskie wkroczyli do domu mieszkańca miasta dokł...
2020-06,zamiast wziąć głęboki oddech i policzyć do dziesięciu aby się uspokoić wybrał wariant najgłupszy z możliwych z zamościa którego do szewskiej pasj...
2020-07,tęgo popił chciał więc nie gorzej podjeść kierowca elektrycznego meleksa pędził po puławach chodnikami i ścieżkami rowerowymi do miejscowego baru...
2020-08,kostry potężna wściekłość i chęć ukarania mężczyzny który krzywdził jego siostrę dosłownie oślepiła piotra n l kiedy spotkał go wieczorem pod skl...


In [82]:
names = list(data.keys())

In [84]:
data_df['publication_month'] = names

In [85]:
data_df

,transcript,publication_month
2020-01,['Prawie trzy tysiące e-zwolnień wystawiają dziennie lekarze z Lubelszczyzny. Nowy kreator do ich wypełniania od Zakładu Ubezpieczeń Społecznych m...,2020-01
2020-02,['Dobre informacje z lubelskiego lotniska. W czerwcu ruszy nowe połączenie z Portu Lotniczego Lublin. Ze Świdnika polecimy do Kolonii/Bonn. Połącz...,2020-02
2020-03,"['To ten sam mężczyzna, u którego wykryto chorobę we wtorek. Do tej pory przebywał w szpitalu w Bełżycach, ale w czwartek przetransportowano go do...",2020-03
2020-04,['Ministerstwo Zdrowia podało w środę przed południem nowe dane z frontu walki z epidemią koronawirusa w Polsce! Pięć ofiar śmiertelnych i ponad 2...,2020-04
2020-05,"['Brał właśnie do rąk szkło, by skosztować produkty destylacji... Policjanci z Białej Podlaskiej (woj. lubelskie) wkroczyli do domu 62-letniego mi...",2020-05
2020-06,"['Zamiast wziąć głęboki oddech i policzyć do dziesięciu, aby się uspokoić, wybrał wariant najgłupszy z możliwych. 21-latek z Zamościa, którego do ...",2020-06
2020-07,"['Tęgo popił, chciał więc nie gorzej podjeść! 49-letni kierowca elektrycznego meleksa pędził po Puławach chodnikami i ścieżkami rowerowymi do miej...",2020-07
2020-08,"['Kostry. Potężna wściekłość i chęć ukarania mężczyzny, który krzywdził jego siostrę, dosłownie oślepiła Piotra N. (21 l.). Kiedy spotkał go wiecz...",2020-08


In [86]:
data_df.to_pickle("corpus.pkl")

In [96]:
stop_words = frozenset(["a", "aby", "adblock","ale", "artykuł" "aż",
                        "bo", "by",
                        "co", "czy",
                        "do", "dla",
                        "gdy", "gdyż", "gdzie",
                        "i", "ich", "im",
                        "jak", "jakże", "jest", "jeśli", "jeżeli",
                        "kto", "ktoś", "która", "które","który",
                        "lub",
                        "ma", "mam", "może",
                        "na", "nad",
                        "o", "od", "oraz",
                        "pod", "polecany artykuł", "przez", "przy",
                        "są", "się",
                        "to", "tym",
                        "u",
                        "w",
                        "z", "że", "żeby"])

In [97]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words=stop_words)
data_cv = cv.fit_transform(data_clean.transcript)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

/home/adsum/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['artykuł', 'polecany'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


,aarts,ab,abigail,abisyńskaco,abisyńskie,abiturientów,abonament,abonamentowe,abonamentu,abonamenty,...,żądał,żądań,żądnego,żłobka,żłobkach,żłobkami,żłobki,żłobkowe,żłobków,żłopać
2020-01,0,0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,1
2020-02,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2020-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,2,0,0,0
2020-04,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,4,0,6,0
2020-05,0,1,0,0,0,0,0,0,0,0,...,1,1,0,2,1,1,4,0,6,0
2020-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
2020-07,1,0,0,0,0,0,0,1,2,0,...,2,0,0,0,0,0,1,0,1,0
2020-08,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [98]:
data_dtm.to_pickle("dtm.pkl")

In [99]:
data_clean.to_pickle('data_clean.pkl')
pickle.dump(cv, open("cv.pkl", "wb"))